# Performance optimization exercise 1

Optimize the following code.

(The type and size of the input is fixed/may not be changed.)

In [74]:
function work!(A, N)
    D = zeros(N,N)
    for i in 1:N
        D = b[i]*c*A
        b[i] = sum(D)
    end
end

N = 100
A = rand(N,N)
b = rand(N)
c = 1.23

work!(A,N)

In [75]:
using BenchmarkTools
@btime work!($A, $N);

  2.796 ms (502 allocations: 7.72 MiB)


## Optimizations

### Avoiding globals

In [76]:
@code_warntype work!(A,N)

Variables
  #self#::Core.Compiler.Const(work!, false)
  A::Array{Float64,2}
  N::Int64
  D::Any
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Nothing
1 ─       (D = Main.zeros(N, N))
│   %2  = (1:N)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = Base.getindex(Main.b, i)::Any
│         (D = %10 * Main.c * A)
│   %12 = Main.sum(D)::Any
│         Base.setindex!(Main.b, %12, i)
│         (@_5 = Base.iterate(%2, %9))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return


In [77]:
function work1!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        D = b[i]*A
        b[i] = sum(D)
    end
end

work1! (generic function with 2 methods)

In [78]:
@code_warntype work1!(A,N,b,c)

Variables
  #self#::Core.Compiler.Const(work1!, false)
  A::Array{Float64,2}
  N::Int64
  b::Array{Float64,1}
  c::Float64
  D::Array{Float64,2}
  @_7::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Nothing
1 ─       (D = Main.zeros(N, N))
│   %2  = (1:N)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_7 = Base.iterate(%2))
│   %4  = (@_7 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_7::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = Base.getindex(b, i)::Float64
│         (D = %10 * A)
│   %12 = Main.sum(D)::Float64
│         Base.setindex!(b, %12, i)
│         (@_7 = Base.iterate(%2, %9))
│   %15 = (@_7 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return


In [79]:
@btime work1!($A, $N, $b, $c);

  2.774 ms (202 allocations: 7.71 MiB)


### Avoiding globals + temporary allocations

In [46]:
function work2!(A, N, b)
    D = zeros(N,N)
    for i in 1:N
        @. D = b[i]*A
        b[i] = sum(D)
    end
end

@btime work2!($A, $N, $b);

  258.407 μs (2 allocations: 78.20 KiB)


In [47]:
function work3!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        @inbounds for j in eachindex(D)
            D[j] = b[i]*c*A[j]
        end
        b[i] = sum(D)
    end
end

@btime work3!($A, $N, $b, $c);

  236.812 μs (2 allocations: 78.20 KiB)


### Avoiding globals + temporary allocations and merging `sum` with loop

In [50]:
function work4!(A, N, b, c)
    D = zeros(N,N)
    for i in 1:N
        s = 0.0
        @inbounds @simd for j in eachindex(D)
            D[j] = b[i]*c*A[j]
            s += D[j]
        end
        b[i] = s
    end
end

@btime work4!($A, $N, $b, $c);

  148.491 μs (2 allocations: 78.20 KiB)
